In [157]:
import pandas as pd
import networkx as nx
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import joblib
import os

In [185]:
mapping_table = {
    'IntGW-01+link-tr-intgw-a-1': 'IntGW-01_Gi2',
    'IntGW-01+link-tr-intgw-b-1': 'IntGW-01_Gi3',
    'IntGW-01+link-intgw-exgw-a-1': 'IntGW-01_Gi5',
    'IntGW-01+link-intgw-exgw-a-2': 'IntGW-01_Gi6',
    'IntGW-01+link-intgw-exgw-a-3': 'IntGW-01_Gi7',
    'IntGW-02+link-tr-intgw-a-2': 'IntGW-02_Gi2',
    'IntGW-02+link-tr-intgw-b-2': 'IntGW-02_Gi3',
    'IntGW-02+link-intgw-exgw-b-1': 'IntGW-02_Gi5',
    'IntGW-02+link-intgw-exgw-b-2': 'IntGW-02_Gi6',
    'IntGW-02+link-intgw-exgw-b-3': 'IntGW-02_Gi7',
    'RR-01+link-tr-intgw-a-3': 'RR-01_Gi2',
    'RR-01+link-tr-intgw-b-3': 'RR-01_Gi3',
    'TR-01+link-tr-intgw-a-1': 'TR-01_Gi2',
    'TR-01+link-tr-intgw-a-2': 'TR-01_Gi3',
    'TR-01+link-tr-intgw-a-3': 'TR-01_Gi4',
    'TR-01+link-tr-tr-a-1': 'TR-01_Gi5',
    'TR-01+link-tr-ssm-a-1': 'TR-01_Gi6',
    'TR-02+link-tr-intgw-b-1': 'TR-02_Gi2',
    'TR-02+link-tr-intgw-b-2': 'TR-02_Gi3',
    'TR-02+link-tr-intgw-b-3': 'TR-02_Gi4',
    'TR-02+link-tr-tr-a-1': 'TR-02_Gi5',
    'TR-02+link-tr-ssm-b-1': 'TR-02_Gi6',
    'timestamp': 'timestamp',
}
metrics = ["admin-status", "tx-pps", "rx-pps", "network-incoming-packets-rate", "network-outgoing-packets-rate"]



In [186]:
# path = "../dataset/train"
# for metric in metrics:
#     df = pd.read_csv(os.path.join(path, metric + ".tsv"), index_col=0, sep="\t")
#     df.columns = df.columns.map(mapping_table)
#     df.to_csv(os.path.join(path, metric + ".tsv"), sep="\t")
# path = "../dataset/test"
# for metric in metrics:
#     df = pd.read_csv(os.path.join(path, metric + ".tsv"), index_col=0, sep="\t")
#     df.columns = df.columns.map(mapping_table)
#     df.to_csv(os.path.join(path, metric + ".tsv"), sep="\t")

In [193]:

metrics = ["cpu-util", "tx-pps", "rx-pps", "network-incoming-packets-rate", "network-outgoing-packets-rate", "prefix-activity-received-current-prefixes"]
link_list = [
    ['IntGW-01_Gi2', 'TR-01_Gi2'],
    ['IntGW-01_Gi3', 'TR-02_Gi2'],
    ['IntGW-02_Gi2', 'TR-01_Gi3'],
    ['IntGW-02_Gi3', 'TR-02_Gi3'],
    ['RR-01_Gi2', 'TR-01_Gi4'],
    ['RR-01_Gi3', 'TR-02_Gi4'],
    ['TR-01_Gi5', 'TR-02_Gi5'],
    ['IntGW-01_Gi2', 'IntGW-01_Gi3'],
    ['IntGW-01_Gi2', 'IntGW-01_Gi5'],
    ['IntGW-01_Gi2', 'IntGW-01_Gi6'],
    ['IntGW-01_Gi2', 'IntGW-01_Gi7'],
    ['IntGW-01_Gi3', 'IntGW-01_Gi5'],
    ['IntGW-01_Gi3', 'IntGW-01_Gi6'],
    ['IntGW-01_Gi3', 'IntGW-01_Gi7'],
    ['IntGW-01_Gi5', 'IntGW-01_Gi6'],
    ['IntGW-01_Gi5', 'IntGW-01_Gi7'],
    ['IntGW-01_Gi6', 'IntGW-01_Gi7'],
    ['IntGW-02_Gi2', 'IntGW-02_Gi3'],
    ['IntGW-02_Gi2', 'IntGW-02_Gi5'],
    ['IntGW-02_Gi2', 'IntGW-02_Gi6'],
    ['IntGW-02_Gi2', 'IntGW-02_Gi7'],
    ['IntGW-02_Gi3', 'IntGW-02_Gi5'],
    ['IntGW-02_Gi3', 'IntGW-02_Gi6'],
    ['IntGW-02_Gi3', 'IntGW-02_Gi7'],
    ['IntGW-02_Gi5', 'IntGW-02_Gi6'],
    ['IntGW-02_Gi5', 'IntGW-02_Gi7'],
    ['IntGW-02_Gi6', 'IntGW-02_Gi7'],
    ['RR-01_Gi2', 'RR-01_Gi3'],
    ['TR-01_Gi2', 'TR-01_Gi3'],
    ['TR-01_Gi2', 'TR-01_Gi4'],
    ['TR-01_Gi2', 'TR-01_Gi5'],
    ['TR-01_Gi2', 'TR-01_Gi6'],
    ['TR-01_Gi3', 'TR-01_Gi4'],
    ['TR-01_Gi3', 'TR-01_Gi5'],
    ['TR-01_Gi3', 'TR-01_Gi6'],
    ['TR-01_Gi4', 'TR-01_Gi5'],
    ['TR-01_Gi4', 'TR-01_Gi6'],
    ['TR-01_Gi5', 'TR-01_Gi6'],
    ['TR-02_Gi2', 'TR-02_Gi3'],
    ['TR-02_Gi2', 'TR-02_Gi4'],
    ['TR-02_Gi2', 'TR-02_Gi5'],
    ['TR-02_Gi2', 'TR-02_Gi6'],
    ['TR-02_Gi3', 'TR-02_Gi4'],
    ['TR-02_Gi3', 'TR-02_Gi5'],
    ['TR-02_Gi3', 'TR-02_Gi6'],
    ['TR-02_Gi4', 'TR-02_Gi5'],
    ['TR-02_Gi4', 'TR-02_Gi6'],
    ['TR-02_Gi5', 'TR-02_Gi6'],
]


In [251]:
class Preprocessing:
    def __init__(self, is_train=True):
        self.transformer = dict()
        self.is_train = is_train
        if not self.is_train:
            self.transformer = self.load()

    def __call__(self, df, is_timeseries=False):
        if self.is_train:
            return self.fit_transform(df, metric)
        else:
            df[:] = self.transformer[metric].transform(df)
            return df

    def fit_transform(self, df, metric):
        if metric == 'cpu-util':
            self.transformer[metric] = MinMaxScaler()
        else:
            self.transformer[metric] = StandardScaler()
        df[:] = self.transformer[metric].fit_transform(df)
        self.dump()
        return df
    
    def transform(self, df):
        df[:] = self.transformer[metric].transform(df)
        return df

    def dump(self, filename='/tmp/gcn_transfomer.bin'):
        with open(filename, 'wb') as f:
            joblib.dump(self.transformer, f)

    def load(self, filename='/tmp/gcn_transfomer.bin'):
        with open(filename, 'rb') as f:
            data = joblib.load(f)
        return data


In [256]:
transformer = Preprocessing(is_train=True)
path = "../dataset/train"
filepath = "../dataset/train/network/raw"
os.makedirs(filepath, exist_ok=True)
interface_df = []
labels = pd.read_csv(os.path.join(path, "label.tsv"), sep="\t", index_col=0)
for metric in metrics:
    df = pd.read_csv(os.path.join(path, metric + ".tsv"), sep="\t", index_col=0)
    df = df.fillna(0)
    df = df.sort_values("timestamp")
    df['_timestamp'] = df['timestamp']
    df = df.set_index("timestamp")
    timestamp = df['_timestamp']
    df.rename(columns={'index': 'timestamp'}, inplace=True)
#     if metric == "prefix-activity-received-current-prefixes":
#         df = df.replace(to_replace=0, method='ffill')
    df = transformer(df, metric)
    df["_timestamp"] = timestamp
    df.index = metric + '_' + df.index.astype(str)
    interface_df.append(df)
transformer.dump()
interface_df = pd.concat(interface_df, sort=True).fillna(0)
grouping_interface_df = interface_df.groupby('_timestamp')
for timestamp in tqdm(interface_df["_timestamp"].unique()):
    tdf = grouping_interface_df.get_group(timestamp)
    tdf.index = tdf.index.map(lambda x: x.split('_')[0])
    tdf = tdf.drop('_timestamp', axis=1)
    edge_list = pd.DataFrame(link_list, columns=['node_from', 'node_to'])
    g = nx.from_pandas_edgelist(edge_list, source='node_from', target='node_to')
    nx.set_node_attributes(g, values={column: {'x': tdf[column].values} for column in tdf.columns})
    nx.set_node_attributes(g, values={column: {'label': labels[labels["timestamp"] == timestamp]["status"].values[0]} for column in tdf.columns})
    nx.write_gpickle(g, os.path.join(filepath, f'network_{timestamp}.pkl'))



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5969/5969 [00:43<00:00, 136.55it/s]


In [253]:
g.nodes(data=True)

NodeDataView({'IntGW-01_Gi2': {'x': array([ 0.88898428, -1.04330483,  0.17153397,  0.19159075, -4.93972442,
        0.        ]), 'label': 0.0}, 'TR-01_Gi2': {'x': array([ 0.74797273,  0.14873886, -0.93220139, -4.65073025,  0.11856197,
        0.        ]), 'label': 0.0}, 'IntGW-01_Gi3': {'x': array([ 0.88898428, -0.92068786,  0.10392936,  0.13594317, -4.90742306,
        0.        ]), 'label': 0.0}, 'TR-02_Gi2': {'x': array([ 0.73171537,  0.0846755 , -0.71443527, -4.63673833,  0.05455992,
        0.        ]), 'label': 0.0}, 'IntGW-02_Gi2': {'x': array([ 8.99566414e-01, -6.15376154e-01,  4.29772582e-03,  4.15067491e-02,
       -4.30512236e+00,  0.00000000e+00]), 'label': 0.0}, 'TR-01_Gi3': {'x': array([ 0.74797273, -0.01646617, -0.46895569, -4.08337807, -0.04739447,
        0.        ]), 'label': 0.0}, 'IntGW-02_Gi3': {'x': array([ 0.89956641, -0.53872429,  0.07562502,  0.10089748, -4.28276076,
        0.        ]), 'label': 0.0}, 'TR-02_Gi3': {'x': array([ 0.73171537,  0.05471106, -0

In [254]:
transformer = Preprocessing(is_train=False)
path = "../dataset/test"
filepath = "../dataset/test/network/raw"
os.makedirs(filepath, exist_ok=True)
interface_df = []
labels = pd.read_csv(os.path.join(path, "label.tsv"), sep="\t", index_col=0)
for metric in metrics:
    df = pd.read_csv(os.path.join(path, metric + ".tsv"), sep="\t", index_col=0)
    df = df.fillna(0)
    df = df.sort_values("timestamp")
    df['_timestamp'] = df['timestamp']
    df = df.set_index("timestamp")
    timestamp = df['_timestamp']
    df = transformer(df, metric)
#     df.rename(columns={'index': 'timestamp'}, inplace=True)
    df["_timestamp"] = timestamp
    df.index = metric + '_' + df.index.astype(str)
    interface_df.append(df)
interface_df = pd.concat(interface_df, sort=True).fillna(0)
grouping_interface_df = interface_df.groupby('_timestamp')
for timestamp in tqdm(interface_df["_timestamp"].unique()):
    tdf = grouping_interface_df.get_group(timestamp)
    tdf.index = tdf.index.map(lambda x: x.split('_')[0])
    tdf = tdf.drop('_timestamp', axis=1)
    edge_list = pd.DataFrame(link_list, columns=['node_from', 'node_to'])
    g = nx.from_pandas_edgelist(edge_list, source='node_from', target='node_to')
    nx.set_node_attributes(g, values={column: {'x': tdf[column].values} for column in tdf.columns})
    nx.set_node_attributes(g, values={column: {'label': labels[labels["timestamp"] == timestamp]["status"].values[0]} for column in tdf.columns})
#     nx.set_node_attributes(g, {column: {index: tdf[column][index] for index in tdf.index} for column in tdf.columns})
    nx.write_gpickle(g, os.path.join(filepath, f'network_{timestamp}.pkl'))


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5333/5333 [00:38<00:00, 137.80it/s]


In [255]:
g.nodes(data=True)

NodeDataView({'IntGW-01_Gi2': {'x': array([ 0.91868645, -0.20886033, -0.28807485,  0.0066077 , -0.70849993,
        0.        ]), 'label': 0.0}, 'TR-01_Gi2': {'x': array([ 0.8854244 , -1.22081585, -0.82499477, -0.66762323,  0.02211057,
        0.        ]), 'label': 0.0}, 'IntGW-01_Gi3': {'x': array([ 0.91868645, -0.22781   , -0.18680265,  0.14881131, -0.74999216,
        0.        ]), 'label': 0.0}, 'TR-02_Gi2': {'x': array([ 0.8585177 , -0.01083696, -0.04218879, -0.7020638 ,  0.16469891,
        0.        ]), 'label': 0.0}, 'IntGW-02_Gi2': {'x': array([ 0.93352338, -0.41194777, -0.40743322,  0.14808939,  0.14391982,
        0.        ]), 'label': 0.0}, 'TR-01_Gi3': {'x': array([ 0.8854244 , -1.11210405, -0.82755449,  0.14715571,  0.16375442,
        0.        ]), 'label': 0.0}, 'IntGW-02_Gi3': {'x': array([ 0.93352338, -0.32805295, -0.47514678,  0.06213301,  0.2219234 ,
        0.        ]), 'label': 0.0}, 'TR-02_Gi3': {'x': array([ 0.8585177 , -0.04691157,  0.04891627,  0.2123504 , 

In [243]:
G = nx.read_gpickle("../dataset/test/network/raw/network_20200707102100.pkl")
G.nodes(data=True)

NodeDataView({'IntGW-01_Gi2': {'x': array([  63.99857645, 2338.        , 2342.        , 2340.53686429,
       2331.95857254,    0.        ]), 'label': 0.0}, 'TR-01_Gi2': {'x': array([  56.01540335, 2344.        , 2312.        , 2327.98203106,
       2342.84515192,    0.        ]), 'label': 0.0}, 'IntGW-01_Gi3': {'x': array([  63.99857645, 2358.        , 2351.        , 2346.75607969,
       2350.60225589,    0.        ]), 'label': 0.0}, 'TR-02_Gi2': {'x': array([  52.45252734, 2352.        , 2367.        , 2348.01987748,
       2349.22062579,    0.        ]), 'label': 0.0}, 'IntGW-02_Gi2': {'x': array([  64.53134295, 2357.        , 2342.        , 2337.02816255,
       2336.48632421,    0.        ]), 'label': 0.0}, 'TR-01_Gi3': {'x': array([  56.01540335, 2342.        , 2360.        , 2334.50056699,
       2339.24958442,    0.        ]), 'label': 0.0}, 'IntGW-02_Gi3': {'x': array([  64.53134295, 2375.        , 2335.        , 2330.79231823,
       2355.04677684,    0.        ]), 'label': 

In [219]:
G = nx.read_gpickle("../dataset/test/network/raw/network_20200713112000.pkl")
G.nodes(data=True)

NodeDataView({'IntGW-01_Gi2': {'x': array([-4.80555502, -0.17897215, -0.26658546, -0.0559486 , -0.70849993,
        0.        ]), 'label': 0.0}, 'TR-01_Gi2': {'x': array([-6.32579471, -1.1234861 , -1.0158905 , -0.91526629,  0.02211057,
        0.        ]), 'label': 0.0}, 'IntGW-01_Gi3': {'x': array([-4.77207202, -0.21062647, -0.0662688 ,  0.18856513, -0.74999216,
        0.        ]), 'label': 0.0}, 'TR-02_Gi2': {'x': array([-6.35053855e+00, -2.62451932e-03, -1.93019541e-01, -1.12424063e+00,
        1.64698911e-01,  0.00000000e+00]), 'label': 0.0}, 'IntGW-02_Gi2': {'x': array([-4.72559232, -0.38327196, -0.22937323,  0.17850224,  0.14391982,
        0.        ]), 'label': 0.0}, 'TR-01_Gi3': {'x': array([-6.48317134, -1.03499333, -1.22185386,  0.04018753,  0.16375442,
        0.        ]), 'label': 0.0}, 'IntGW-02_Gi3': {'x': array([-4.71102517, -0.30744213, -0.31167894,  0.08823419,  0.2219234 ,
        0.        ]), 'label': 0.0}, 'TR-02_Gi3': {'x': array([-6.40485531, -0.03363241, -0